<a href="https://colab.research.google.com/github/hthomas229/PurpleCrown/blob/main/polars_tutorial_videos_1_to_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#POLARS TUTORIAL

##Intro to Polars and Your First DataFrame

🐻‍❄️ Install and import polars


In [ ]:
!pip install polars

In [ ]:
import polars as pl
from rich import print #for pretty printing)

Create a polars series

In [ ]:
series = pl.Series("fruits", ["apple", "banana", "cherry"])
print(series)

Create a Polars Data Frame

In [ ]:
df = pl.DataFrame(
    {
        "name" : ["Raj", "Bev", "Ken"],
        "age" : [21, 31, 43],
        "height" : [5.2, 5.1, 6.3],
        "is_employed" : [False, True, True]
    }
)
print(df)

🐻‍❄️ The first big difference between Polars and Pandas becomes obvious here:  NO INDEX.  This means no loc[] and no iloc[].

Polars provides a schema function for a quick glance


In [ ]:
df.collect_schema()

Get a Glimpse of the Data

In [ ]:
df.glimpse()

Other common pandas functions like head and describe work also

In [ ]:
df.head()

In [ ]:
print(df.describe())

##Select Context Operations on DataFrames

🐻‍❄️ Note:  Polars is immutable. When we create **result** below, it returns a new dataframe.  There is no in place modification.

Select Specific Columns

In [ ]:
print(df)

In [ ]:
result = df.select("name", "age")
print(result)


Rename Columns with Alias

In [ ]:
#rename is_employed column to working? select name and working columns
result = df.select(
    "name",
    pl.col("is_employed").alias("working?")
)
print(result)

Perform Basic Calculations

In [ ]:
#double the age column; display only name and age
result = df.select(
    "name",
    pl.col("age") * 2
)
print(result)

Derived Column in Select

In [ ]:
#add a boolean column called tall; True if height > 6.0; display name, height and tall
result = df.select(
    "name",
    "height",
    (pl.col("height") > 6.0).alias("tall")
)
print(result)

Create a Literal (Constant Column)

In [ ]:
#create a column called value whose only value is the string name; selct name & value
result = df.select(
    pl.col("name"),
    pl.lit("name").alias("value")
)

print(result)

##Filtering Polars DataFrames


🐻‍❄️For the next few tutorials we will be using pokemon data. I will place the data file in my github repository https://github.com/hthomas229/PurpleCrown. Download it and upload to CoLab each time you run the notebook.

Scan Data from CSV - Lazy

🐻‍❄️ Lazy evaluation is one of the most powerful features of Polars.  We will cover it in detail in later tutorials.

In [ ]:
lf =pl.scan_csv("PokemonData.csv")
print(lf)

Read Data from CSV - Eager

🐻‍❄️ Since we are working with a small dataset we will load eager for the next few tutorials.

I will use df for eager and lf for lazy. Also result for eager and query for lazy in my solutions.


In [ ]:
df = pl.read_csv("PokemonData.csv")
print(df.head())

shape: (5, 12)
┌─────┬───────────────────────┬───────┬────────┬───┬───────┬───────┬────────────┬───────────┐
│ Num ┆ Name                  ┆ Type1 ┆ Type2  ┆ … ┆ SpDef ┆ Speed ┆ Generation ┆ Legendary │
│ --- ┆ ---                   ┆ ---   ┆ ---    ┆   ┆ ---   ┆ ---   ┆ ---        ┆ ---       │
│ i64 ┆ str                   ┆ str   ┆ str    ┆   ┆ i64   ┆ i64   ┆ i64        ┆ bool      │
╞═════╪═══════════════════════╪═══════╪════════╪═══╪═══════╪═══════╪════════════╪═══════════╡
│ 1   ┆ Bulbasaur             ┆ Grass ┆ Poison ┆ … ┆ 65    ┆ 45    ┆ 1          ┆ false     │
│ 2   ┆ Ivysaur               ┆ Grass ┆ Poison ┆ … ┆ 80    ┆ 60    ┆ 1          ┆ false     │
│ 3   ┆ Venusaur              ┆ Grass ┆ Poison ┆ … ┆ 100   ┆ 80    ┆ 1          ┆ false     │
│ 3   ┆ VenusaurMega Venusaur ┆ Grass ┆ Poison ┆ … ┆ 120   ┆ 80    ┆ 1          ┆ false     │
│ 4   ┆ Charmander            ┆ Fire  ┆ null   ┆ … ┆ 50    ┆ 65    ┆ 1          ┆ false     │
└─────┴───────────────────────┴───────┴────────┴───┴───────┴───────┴────────────┴───────────┘

Basic Filter

In [ ]:
#Find fast pokemon: Speed > 100
result = df.filter(pl.col("Speed") > 100).select("Name", "Speed")
print(result)

shape: (106, 2)
┌───────────────────────┬───────┐
│ Name                  ┆ Speed │
│ ---                   ┆ ---   │
│ str                   ┆ i64   │
╞═══════════════════════╪═══════╡
│ BeedrillMega Beedrill ┆ 145   │
│ Pidgeot               ┆ 101   │
│ PidgeotMega Pidgeot   ┆ 121   │
│ Raichu                ┆ 110   │
│ Dugtrio               ┆ 120   │
│ …                     ┆ …     │
│ Heliolisk             ┆ 109   │
│ Hawlucha              ┆ 118   │
│ Dedenne               ┆ 101   │
│ Noivern               ┆ 123   │
│ DiancieMega Diancie   ┆ 110   │
└───────────────────────┴───────┘

Multiple Conditions

`Polars uses ==, !=, &, | `

In [ ]:
#find pokemon with Attack > 90 and Defense > 80
result = df.filter(
    (pl.col("Attack") > 90) & (pl.col("Defense") > 80)
).select("Name", "Attack", "Defense")
print(result)

shape: (137, 3)
┌───────────────────────────┬────────┬─────────┐
│ Name                      ┆ Attack ┆ Defense │
│ ---                       ┆ ---    ┆ ---     │
│ str                       ┆ i64    ┆ i64     │
╞═══════════════════════════╪════════╪═════════╡
│ VenusaurMega Venusaur     ┆ 100    ┆ 123     │
│ CharizardMega Charizard X ┆ 130    ┆ 111     │
│ BlastoiseMega Blastoise   ┆ 103    ┆ 120     │
│ Sandslash                 ┆ 100    ┆ 110     │
│ Nidoqueen                 ┆ 92     ┆ 87      │
│ …                         ┆ …      ┆ …       │
│ Yveltal                   ┆ 131    ┆ 95      │
│ Zygarde50% Forme          ┆ 100    ┆ 121     │
│ Diancie                   ┆ 100    ┆ 150     │
│ DiancieMega Diancie       ┆ 160    ┆ 110     │
│ Volcanion                 ┆ 110    ┆ 120     │
└───────────────────────────┴────────┴─────────┘

String Filtering

In [ ]:
#return only Type1 Fire pokemon
result = df.filter(
    pl.col("Type1") == "Fire"
)

print(result)

shape: (52, 12)
┌─────┬───────────────────────────┬───────┬────────┬───┬───────┬───────┬────────────┬───────────┐
│ Num ┆ Name                      ┆ Type1 ┆ Type2  ┆ … ┆ SpDef ┆ Speed ┆ Generation ┆ Legendary │
│ --- ┆ ---                       ┆ ---   ┆ ---    ┆   ┆ ---   ┆ ---   ┆ ---        ┆ ---       │
│ i64 ┆ str                       ┆ str   ┆ str    ┆   ┆ i64   ┆ i64   ┆ i64        ┆ bool      │
╞═════╪═══════════════════════════╪═══════╪════════╪═══╪═══════╪═══════╪════════════╪═══════════╡
│ 4   ┆ Charmander                ┆ Fire  ┆ null   ┆ … ┆ 50    ┆ 65    ┆ 1          ┆ false     │
│ 5   ┆ Charmeleon                ┆ Fire  ┆ null   ┆ … ┆ 65    ┆ 80    ┆ 1          ┆ false     │
│ 6   ┆ Charizard                 ┆ Fire  ┆ Flying ┆ … ┆ 85    ┆ 100   ┆ 1          ┆ false     │
│ 6   ┆ CharizardMega Charizard X ┆ Fire  ┆ Dragon ┆ … ┆ 85    ┆ 100   ┆ 1          ┆ false     │
│ 6   ┆ CharizardMega Charizard Y ┆ Fire  ┆ Flying ┆ … ┆ 115   ┆ 100   ┆ 1          ┆ false     │
│ …   ┆ …                         ┆ …     ┆ …      ┆ … ┆ …     ┆ …     ┆ …          ┆ …         │
│ 662 ┆ Fletchinder               ┆ Fire  ┆ Flying ┆ … ┆ 52    ┆ 84    ┆ 6          ┆ false     │
│ 663 ┆ Talonflame                ┆ Fire  ┆ Flying ┆ … ┆ 69    ┆ 126   ┆ 6          ┆ false     │
│ 667 ┆ Litleo                    ┆ Fire  ┆ Normal ┆ … ┆ 54    ┆ 72    ┆ 6          ┆ false     │
│ 668 ┆ Pyroar                    ┆ Fire  ┆ Normal ┆ … ┆ 66    ┆ 106   ┆ 6          ┆ false     │
│ 721 ┆ Volcanion                 ┆ Fire  ┆ Water  ┆ … ┆ 90    ┆ 70    ┆ 6          ┆ true      │
└─────┴───────────────────────────┴───────┴────────┴───┴───────┴───────┴────────────┴───────────┘

Many Other String Methods



In [ ]:
#return all pokemon whose name contains the string "saur"
result = df.filter(
    pl.col("Name").str.contains("saur")
)
print(result)

shape: (4, 12)
┌─────┬───────────────────────┬───────┬────────┬───┬───────┬───────┬────────────┬───────────┐
│ Num ┆ Name                  ┆ Type1 ┆ Type2  ┆ … ┆ SpDef ┆ Speed ┆ Generation ┆ Legendary │
│ --- ┆ ---                   ┆ ---   ┆ ---    ┆   ┆ ---   ┆ ---   ┆ ---        ┆ ---       │
│ i64 ┆ str                   ┆ str   ┆ str    ┆   ┆ i64   ┆ i64   ┆ i64        ┆ bool      │
╞═════╪═══════════════════════╪═══════╪════════╪═══╪═══════╪═══════╪════════════╪═══════════╡
│ 1   ┆ Bulbasaur             ┆ Grass ┆ Poison ┆ … ┆ 65    ┆ 45    ┆ 1          ┆ false     │
│ 2   ┆ Ivysaur               ┆ Grass ┆ Poison ┆ … ┆ 80    ┆ 60    ┆ 1          ┆ false     │
│ 3   ┆ Venusaur              ┆ Grass ┆ Poison ┆ … ┆ 100   ┆ 80    ┆ 1          ┆ false     │
│ 3   ┆ VenusaurMega Venusaur ┆ Grass ┆ Poison ┆ … ┆ 120   ┆ 80    ┆ 1          ┆ false     │
└─────┴───────────────────────┴───────┴────────┴───┴───────┴───────┴────────────┴───────────┘

There will be a whole tutorial on string methods later.

Boolean Filtering

In [ ]:
#find pokemon who are Legendary or have a Speed higher than 130
result = df.filter(
    pl.col("Legendary") | (pl.col("Speed") >= 130)
).select("Name", "Speed", "Legendary")

print(result)

shape: (78, 3)
┌───────────────────────┬───────┬───────────┐
│ Name                  ┆ Speed ┆ Legendary │
│ ---                   ┆ ---   ┆ ---       │
│ str                   ┆ i64   ┆ bool      │
╞═══════════════════════╪═══════╪═══════════╡
│ BeedrillMega Beedrill ┆ 145   ┆ false     │
│ AlakazamMega Alakazam ┆ 150   ┆ false     │
│ GengarMega Gengar     ┆ 130   ┆ false     │
│ Electrode             ┆ 140   ┆ false     │
│ Jolteon               ┆ 130   ┆ false     │
│ …                     ┆ …     ┆ …         │
│ Diancie               ┆ 50    ┆ true      │
│ DiancieMega Diancie   ┆ 110   ┆ true      │
│ HoopaHoopa Confined   ┆ 70    ┆ true      │
│ HoopaHoopa Unbound    ┆ 80    ┆ true      │
│ Volcanion             ┆ 70    ┆ true      │
└───────────────────────┴───────┴───────────┘